In [1]:
import numpy as np
import scipy as scp
from random import seed
from random import random
from random import randrange
from random import uniform

In [66]:
class NeuralNetwork:
    class BaseNode:
        def __init__(self, outer, function):
            self.outer = outer
            self.function = function
            self.bias = uniform(-1,1)
    class StartNode(BaseNode):
        def __init__(self, outer, function):
            super().__init__(outer, function)
            self.values = []
            self.weightIndexes = []
        def setWeightIndexes(self, weightIndexes):
            self.weightIndexes = weightIndexes
        def setVal(self, val):
            self.values = val
        def getResult(self):
            if not self.values:
                raise Exception('No values')
            else:
                sum = self.bias
                for i in range(len(self.values)):
                    sum = self.outer.genes[self.weightIndexes[i]]*self.values[i]
            return self.function(sum)
        def mutateBias(self, probability):
            if random() < probability:
                self.bias *= uniform(0, 2)
    class Node(BaseNode):
        def __init__(self, outer, function):
            super().__init__(outer, function)
            self.prevNodes = []
            self.weightIndexes = []
        def addPrevNode(self, node):
            self.prevNodes.append(node)
        def addPrevNodes(self, nodes, weightIndexes):
            for n in nodes:
                self.addPrevNode(n)
            self.weightIndexes = weightIndexes
        def getResult(self):
            if not self.prevNodes:
                raise Exception('No previous nodes')
            else:
                sum = self.bias
                for i in range(len(self.prevNodes)):
                    sum += self.outer.genes[self.weightIndexes[i]]*self.prevNodes[i].getResult()
            return self.function(sum)
        def mutateBias(self, probability):
            if random() < probability:
                self.bias *= uniform(0, 2)
            for n in self.prevNodes:
                n.mutateBias(probability)
    def __init__(self, noOfValues, shape, function):
        if noOfValues < 1:
            raise Exception('At least one value is required')
        self.noOfValues = noOfValues
        self.shape = shape
        self.genes = []
        self.startNodes = []
        if not shape:
            for i in range(noOfValues):
                self.genes.append(uniform(-1,1))
            self.startNodes.append(self.StartNode(self, function))
            self.endNode = self.startNodes[0]
            self.endNode.setWeightIndexes(list(range(noOfValues)))
        else:
            for i in range(shape[0]):
                self.startNodes.append(self.StartNode(self, function))
                weightIndexes = list(range(noOfValues))
                length = len(self.genes)-1
                for j in range(noOfValues):
                    weightIndexes[j] += length
                    self.genes.append(uniform(-1,1))
                self.startNodes[i].setWeightIndexes(weightIndexes)
            shape.remove(shape[0])
            prev = self.startNodes
            lastIndex = len(self.genes)-1
            for n in shape:
                if n < 1:
                    raise Exception('No layer can have less than one node')
                tmp = []
                for i in range(n):
                    tmp.append(self.Node(self, function))
                    weightIndexes = list(range(len(prev)))
                    length = len(self.genes)-1
                    for j in range(len(prev)):
                        weightIndexes[j] += length
                        self.genes.append(uniform(-1,1))
                    tmp[i].addPrevNodes(prev, weightIndexes)
                prev = tmp
            self.endNode = self.Node(self, function)
            weightIndexes = list(range(len(prev)))
            length = len(self.genes)-1
            for j in range(len(prev)):
                weightIndexes[j] += length
                self.genes.append(uniform(-1,1))
            self.endNode.addPrevNodes(prev, weightIndexes)
    def getResult(self):
        return self.endNode.getResult()
    def setVal(self, values):
        for n in self.startNodes:
            n.setVal(values)
    def mutate(self, probability):
        for i in range(len(self.genes)):
            if random() < probability:
                self.genes[i] *= uniform(0, 2)
        self.endNode.mutateBias(probability)
    def breed(network1, network2, mutProb):
        if network1.shape != network2.shape or network1.noOfValues != network2.noOfValues:
            raise Exception('Networks have different shapes or functions!')
        for i in range(len(network1.genes)):
            network1.genes[i] += network2.genes[i]
            network1.genes[i] /= 2
        endNode = breed(network1.endNode, network2.endNode)
        network1.endNode = endNode
        for i in range(len(network1.shape)):
            endNode = endNode.prevNodes[0]
        network1.startNodes = endNode.prevNodes
        network1.mutate(mutProb)
        return network1
def breed(node1, node2):
    node1.bias += node2.bias
    node1.bias /= 2
    if isinstance(node1, NeuralNetwork.Node) and isinstance(node2, NeuralNetwork.Node):
        for i in range(len(node1.prevNodes)):
            node1.prevNodes[i] = breed(node1.prevNodes[i], node2.prevNodes[i])
    return node1

False

In [67]:
wat = NeuralNetwork(3, [2, 3, 2], lambda x: x)
wat2 = NeuralNetwork(3, [2, 3 ,2], lambda x: x)
wat3 = NeuralNetwork.breed(wat, wat2, 0.02)
wat3.setVal([1, 7, 101])
print(wat3.getResult())

-7.941808590356683


In [7]:
class a:
    def __init__(self, v):
        self.list = v
    def write(self):
        print(self.list)

c = [1, 2]
myList = a(c)
myList2 = a(myList.list)
myList.write()
c[0] = 4
myList.write()
myList2.write()
c[1] = 5
myList.write()
myList2.write()

[1, 2]
[4, 2]
[4, 2]
[4, 5]
[4, 5]


AttributeError: 'list' object has no attribute 'size'